<a href="https://colab.research.google.com/github/hxnguyen/Tram2Flows/blob/tram2flows_v2/TTPs_to_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Rag Result##

In [1]:
from google.colab import files
import json
uploaded = files.upload()


Saving output_prompt_testing_anu_filtered.json to output_prompt_testing_anu_filtered.json


In [2]:
import pandas as pd
key_ls = list(uploaded.keys())
df = pd.read_json(key_ls[0])

In [3]:
### Eliminate unsued collums
attack_name = df.loc[0]['name']
df = df.drop(['index', 'segment', 'name'], axis = 1)

In [4]:
### Eliminate duplication

from difflib import SequenceMatcher as SM


for i in range(0, len(df.index)):
  if i >= len(df.index):
    break
  cur = df.loc[i]
  drop_ls = []
  ls = []
  ls.append(cur['assets'])
  for j in range(2):
    index = i + j + 1
    if index >= len(df.index):
      continue
    fol = df.loc[index]



    if cur['label(s)'] == fol['label(s)']:
      for a1 in cur['label(s)']:
        for a2 in fol['label(s)']:
          asset_rate = SM(None, a1, a2).ratio()
          if asset_rate < 0.7:
            df.loc[i]['assets'].append(a2)
      for lbl in fol['prerequisite']:
        if lbl not in cur['prerequisite']:
          df.loc[i]['prerequisite'].append(lbl)
      drop_ls.append(index)
  # df.loc[i]['assets'] = ls
  df = df.drop(drop_ls)
  df = df.reset_index(drop=True)


In [5]:
### Eliminate Infinite Loops
def eliminate_loop(attackframe):
  for i in range(len(attackframe.index)):
    for pre in attackframe.loc[i]['prerequisite']:
      if pre == attackframe.loc[i]['label(s)']:
        attackframe.loc[i]['label(s)'].remove(pre)

  return attackframe

df = eliminate_loop(df)
df

,label(s),assets,prerequisite
0,[Spearphishing Attachment - T1566.001],"[ Email, credentials, webshell]",[]
1,[Obfuscated Files or Information - T1027],[ Email],[Spearphishing Attachment - T1566.001]
2,[Malicious File - T1204.002],[ Email attachment],[Spearphishing Attachment - T1566.001]
3,[Ingress Tool Transfer - T1105],[ Attachments.],[Spearphishing Attachment - T1566.001]
4,[Valid Accounts - T1078],[ Administrator credentials],"[Ingress Tool Transfer - T1105, Spearphishing ..."
5,[Spearphishing Attachment - T1566.001],[ Calendar information.],"[Malicious File - T1204.002, Ingress Tool Tran..."
6,[Proxy - T1090],[ TOR exit node],[Valid Accounts - T1078: Administrator credent...
7,[Valid Accounts - T1078],"[ Exploits, stolen credentials]","[Proxy - T1090, Spearphishing Attachment - T15..."
8,[File and Directory Discovery - T1083],[ File shares.],[Proxy - T1090]
9,[Obfuscated Files or Information - T1027],[ ESD file shares],[File and Directory Discovery - T1083]


##Structure Parsing##

In [66]:
def dependency_parsing(attackframe):
  dependency = {}
  for i in range(len(attackframe.index)):
    cur_ttp = attackframe.loc[i]['label(s)'][0] + f' - {i}'
    dependency[cur_ttp] = {}
    dependency[cur_ttp]['assets'] = attackframe.loc[i]['assets'][0].split(", ")
    dependency[cur_ttp]['index'] = i
    dependency[cur_ttp]['prerequisite'] = {}
    for pre in attackframe.loc[i]['prerequisite']:
      for j in range(3):
        index = i - j - 1
        if index < 0:
          continue
        if attackframe.loc[index]['label(s)'][0] == pre:
          dependency[cur_ttp]['prerequisite'][pre] = index


  return dependency

dependency = dependency_parsing(df)
print(dependency['Proxy - T1090 - 6'])

{'assets': [' TOR exit node'], 'index': 6, 'prerequisite': {'Spearphishing Attachment - T1566.001': 5, 'Valid Accounts - T1078': 4}}


##Reformatting JSON##


In [7]:
!pip install datetime
!pip install json
!pip install uuid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 7.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6479 sha256=2c1909508862bd27ea91ca8cb211f0b4bc17c58a410872ad6798e3c69061dcfa
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [67]:
## Add Ids to attack actions and attack assets
import uuid

def add_ids(attackdict):
  for key, value in attackdict.items():
    attackdict[key]["id"] = "attack-action--" + str(uuid.uuid1())
    awid = {}
    for asset in attackdict[key]['assets']:
      awid[asset] = "attack-asset--" + str(uuid.uuid1())
    attackdict[key]['assets'] = awid

  return attackdict

dependency = add_ids(dependency)


{'Spearphishing Attachment - T1566.001 - 0': {'assets': {' Email': 'attack-asset--747bdf96-a059-11ee-b659-0242ac1c000c', 'credentials': 'attack-asset--747be09a-a059-11ee-b659-0242ac1c000c', 'webshell': 'attack-asset--747be14e-a059-11ee-b659-0242ac1c000c'}, 'index': 0, 'prerequisite': {}, 'id': 'attack-action--747bdbfe-a059-11ee-b659-0242ac1c000c'}, 'Obfuscated Files or Information - T1027 - 1': {'assets': {' Email': 'attack-asset--747be2ca-a059-11ee-b659-0242ac1c000c'}, 'index': 1, 'prerequisite': {'Spearphishing Attachment - T1566.001': 0}, 'id': 'attack-action--747be216-a059-11ee-b659-0242ac1c000c'}, 'Malicious File - T1204.002 - 2': {'assets': {' Email attachment': 'attack-asset--747be432-a059-11ee-b659-0242ac1c000c'}, 'index': 2, 'prerequisite': {'Spearphishing Attachment - T1566.001': 0}, 'id': 'attack-action--747be37e-a059-11ee-b659-0242ac1c000c'}, 'Ingress Tool Transfer - T1105 - 3': {'assets': {' Attachments.': 'attack-asset--747be586-a059-11ee-b659-0242ac1c000c'}, 'index': 3, 

In [68]:
## Prerequisite to following attacks

def find_next_attacks(attackdict):
  start_ref = []
  for key, value in attackdict.items():
    attackdict[key]['next_act'] = []
    ls = attackdict[key]['prerequisite']
    if len(ls) == 0:
      start_ref.append(value['id'])
    for prekey, preval in ls.items():
      prerequisite_key = prekey + f' - {preval}'
      attackdict[prerequisite_key]['next_act'].append(value['id'])
    del attackdict[key]['prerequisite']
  return attackdict, start_ref

dependency, start_ref = find_next_attacks(dependency)

{'Spearphishing Attachment - T1566.001 - 0': {'assets': {' Email': 'attack-asset--747bdf96-a059-11ee-b659-0242ac1c000c', 'credentials': 'attack-asset--747be09a-a059-11ee-b659-0242ac1c000c', 'webshell': 'attack-asset--747be14e-a059-11ee-b659-0242ac1c000c'}, 'index': 0, 'id': 'attack-action--747bdbfe-a059-11ee-b659-0242ac1c000c', 'next_act': ['attack-action--747be216-a059-11ee-b659-0242ac1c000c', 'attack-action--747be37e-a059-11ee-b659-0242ac1c000c', 'attack-action--747be4dc-a059-11ee-b659-0242ac1c000c']}, 'Obfuscated Files or Information - T1027 - 1': {'assets': {' Email': 'attack-asset--747be2ca-a059-11ee-b659-0242ac1c000c'}, 'index': 1, 'id': 'attack-action--747be216-a059-11ee-b659-0242ac1c000c', 'next_act': []}, 'Malicious File - T1204.002 - 2': {'assets': {' Email attachment': 'attack-asset--747be432-a059-11ee-b659-0242ac1c000c'}, 'index': 2, 'id': 'attack-action--747be37e-a059-11ee-b659-0242ac1c000c', 'next_act': ['attack-action--747be838-a059-11ee-b659-0242ac1c000c']}, 'Ingress To

##TEMPLATEs##

##Converting

In [69]:
## converting to JSON-like attackflow
from datetime import datetime

def convertDictKeys2Nodes(attackdict, extension):
  Nodes = []
  cd = datetime.now()
  current_date = cd.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
  for key,value in attackdict.items():
    name = key.split("-")[0].strip()
    tech_id = key.split("-")[1].strip()

    action = {
      "type": "attack-action",
      "id": value['id'],
      "spec_version": "2.1",
      "created": current_date,
      "modified": current_date,
      "extensions": extension,
      "name": name,
      "tactic_id": tech_id,
      "confidence": 100,
      "effect_refs": value['next_act'],
      "asset_refs": list(value['assets'].values())
    }
    if len(action['effect_refs']) == 0:
      del action['effect_refs']
    if len(action['asset_refs']) == 0:
      del action['asset_refs']

    Nodes.append(action)
    for akey, aval in value['assets'].items():
      asset = {
        "type": "attack-asset",
        "id": aval,
        "spec_version": "2.1",
        "created": current_date,
        "modified": current_date,
        "extensions": extension,
        "name": akey
      }
      Nodes.append(asset)
  return Nodes


def converting2attackflow(attackdict, start_ref):
  cd = datetime.now()
  current_date = cd.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
  bundle_id = "bundle--" + str(uuid.uuid1())
  extension_def_id = "extension-definition--fb9c968a-745b-4ade-9b25-c324172197f4"
  af_id = "attack-flow--" + str(uuid.uuid1())

  #Outer node
  attack_flow_bundle = {
    "type": "bundle",
    "id": bundle_id,
    "spec_version": "2.1",
    "created": current_date,
    "modified": current_date,
    "objects": []
  }

  ## extension definition
  extension_def = {
      "type": "extension-definition",
      "id": extension_def_id,
      "spec_version": "2.1",
      "created": "2022-08-02T19:34:35.143Z",
      "modified": "2022-08-02T19:34:35.143Z",
      "name": "Attack Flow",
      "description": "Extends STIX 2.1 with features to create Attack Flows.",
      "created_by_ref": "identity--fb9c968a-745b-4ade-9b25-c324172197f4",
      "schema": "https://center-for-threat-informed-defense.github.io/attack-flow/stix/attack-flow-schema-2.0.0.json",
      "version": "2.0.0",
      "extension_types": [
        "new-sdo"
      ],
      "external_references": [
        {
          "source_name": "Documentation",
          "description": "Documentation for Attack Flow",
          "url": "https://center-for-threat-informed-defense.github.io/attack-flow"
        },
        {
          "source_name": "GitHub",
          "description": "Source code repository for Attack Flow",
          "url": "https://github.com/center-for-threat-informed-defense/attack-flow"
        }
      ]
  }
  attack_flow_bundle['objects'].append(extension_def)

  extension = {
        f"{extension_def_id}": {
          "extension_type": "new-sdo"
        }
      }

  ##identity
  identity = {
      "type": "identity",
      "id": "identity--fb9c968a-745b-4ade-9b25-c324172197f4",
      "spec_version": "2.1",
      "created": "2022-08-02T19:34:35.143Z",
      "modified": "2022-08-02T19:34:35.143Z",
      "create_by_ref": "identity--fb9c968a-745b-4ade-9b25-c324172197f4",
      "name": "MITRE Engenuity Center for Threat-Informed Defense",
      "identity_class": "organization"
  }
  attack_flow_bundle['objects'].append(identity)

  ##identity_me
  my_id = "identity--" + str(uuid.uuid1())
  id_me = {
      "type": "identity",
      "id": my_id,
      "spec_version": "2.1",
      "created": "2023-06-27T01:41:48.030Z",
      "modified": "2023-06-27T01:41:48.030Z",
      "name": "CaChepOnDua"
  }

  ## attack flow element
  attack_flow = {
      "type": "attack-flow",
      "id": af_id,
      "spec_version": "2.1",
      "created": current_date,
      "modified": current_date,
      "extensions": extension,
      "created_by_ref": my_id,
      "start_refs": start_ref,
      "name": attack_name,
      "scope": "incident"
  }

  attack_flow_bundle['objects'].append(attack_flow)
  attack_flow_bundle['objects'].append(id_me)
  ## other elements
  actions_and_assets = convertDictKeys2Nodes(attackdict, extension)
  print(actions_and_assets)
  attack_flow_bundle["objects"] = attack_flow_bundle["objects"] + actions_and_assets

  return attack_flow_bundle

attack_flow = converting2attackflow(dependency, start_ref)


print(attack_flow)


[{'type': 'attack-action', 'id': 'attack-action--747bdbfe-a059-11ee-b659-0242ac1c000c', 'spec_version': '2.1', 'created': '2023-12-21T23:34:33.795875Z', 'modified': '2023-12-21T23:34:33.795875Z', 'extensions': {'extension-definition--fb9c968a-745b-4ade-9b25-c324172197f4': {'extension_type': 'new-sdo'}}, 'name': 'Spearphishing Attachment', 'tactic_id': 'T1566.001', 'confidence': 100, 'effect_refs': ['attack-action--747be216-a059-11ee-b659-0242ac1c000c', 'attack-action--747be37e-a059-11ee-b659-0242ac1c000c', 'attack-action--747be4dc-a059-11ee-b659-0242ac1c000c'], 'asset_refs': ['attack-asset--747bdf96-a059-11ee-b659-0242ac1c000c', 'attack-asset--747be09a-a059-11ee-b659-0242ac1c000c', 'attack-asset--747be14e-a059-11ee-b659-0242ac1c000c']}, {'type': 'attack-asset', 'id': 'attack-asset--747bdf96-a059-11ee-b659-0242ac1c000c', 'spec_version': '2.1', 'created': '2023-12-21T23:34:33.795875Z', 'modified': '2023-12-21T23:34:33.795875Z', 'extensions': {'extension-definition--fb9c968a-745b-4ade-9b2

##Download file##

In [70]:
from google.colab import files
import json

with open('ANU1.json', 'w') as file:
    json.dump(attack_flow, file)

files.download('ANU1.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>